In [1]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping


Using TensorFlow backend.
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/congyu/.virtua

In [2]:
from cnn.cnn import load_dataset, create_model, steps
from dataset import dataset, load_files
from utilities.labels import LABELS
from dataset.data_augmentation import augmentated_filenames2

In [3]:
data_dir = "./data/mit_bih/"
save_dir = "./data/beats_img/"
_dataset_dir = './data/beats_img'
_model = './trained_models/cnn_baseline.h5'

In [4]:
train = None
validation = None
augmentation = False
_batch_size = 32
_size = (64, 64)
_epochs = 30

In [5]:
model = create_model()

train, validation, test = dataset.load_files(_dataset_dir, 
                                             verbose=True, 
                                             keep_ratio=0.1)
    
print(len(train))
print(len(validation))
print(len(test))


load tvt from ./data/beats_img
7533
1614
1614


In [ ]:
if augmentation:
    train = augmentated_filenames2(train)

callbacks_list = [ModelCheckpoint(_model, monitor='val_loss', save_best_only=False),
                  EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto',
                                baseline=None, restore_best_weights=False),
                  ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, verbose=0, mode='auto')
# TrainValTensorBoard(write_graph=False)
]

data_gen_train = load_dataset(train, _dataset_dir,
                              _batch_size, _size,
                              random_crop=augmentation,
                              random_rotate=augmentation,
                              flip=augmentation)

data_gen_valid = load_dataset(validation, _dataset_dir,
                              _batch_size, _size,
                              random_crop=augmentation,
                              random_rotate=augmentation,
                              flip=augmentation)

model.fit_generator(data_gen_train,
                    steps_per_epoch=steps(train, _batch_size),
                    epochs=_epochs,
                    validation_data=data_gen_valid,
                    validation_steps=steps(validation, _batch_size),
                    callbacks=callbacks_list)


Epoch 1/30
236/235 [==============================] - 20s 84ms/step - loss: 1.1105 - accuracy: 0.7540 - val_loss: 1.6374 - val_accuracy: 0.6797
Epoch 2/30
236/235 [==============================] - 18s 78ms/step - loss: 0.5025 - accuracy: 0.8889 - val_loss: 1.3310 - val_accuracy: 0.8631
Epoch 3/30
236/235 [==============================] - 18s 75ms/step - loss: 0.3532 - accuracy: 0.9327 - val_loss: 0.9509 - val_accuracy: 0.8612
Epoch 4/30
236/235 [==============================] - 18s 78ms/step - loss: 0.2940 - accuracy: 0.9498 - val_loss: 0.6912 - val_accuracy: 0.9244
Epoch 5/30
236/235 [==============================] - 18s 77ms/step - loss: 0.2602 - accuracy: 0.9559 - val_loss: 1.1694 - val_accuracy: 0.7937
Epoch 6/30
236/235 [==============================] - 18s 77ms/step - loss: 0.2417 - accuracy: 0.9618 - val_loss: 1.1424 - val_accuracy: 0.7869
Epoch 7/30
236/235 [==============================] - 18s 77ms/step - loss: 0.2136 - accuracy: 0.9701 - val_loss: 0.3280 - val_accuracy

In [ ]:
from evaluation import evaluate_cnn

In [ ]:
evaluate_cnn(model, keep_ratio=0.1)